In [1]:
import os

# 'models' 폴더가 있는지 확인
models_path = "/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/models"
print(os.path.exists(models_path))  # True면 존재, False면 없음

# 폴더 내부 파일 확인
if os.path.exists(models_path):
    print(os.listdir(models_path))  # 'fast_scnn.py'가 있는지 확인

True
['.ipynb_checkpoints', 'bisenetv2.py', '__pycache__', 'modules.py', '__init__.py', 'model_registry.py', 'BiSeNet_V2.py', 'fastscnn.py', 'fast_scnn.py']


In [2]:
import sys

# Python 모듈 경로에 'models' 폴더 추가
models_path = "/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/models"
utils_path = "/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/utils"

if models_path not in sys.path:
    sys.path.append(models_path)  # models 폴더 추가

if utils_path not in sys.path:
    sys.path.append(utils_path)  # utils 폴더 추가

# sys.path 확인 (제대로 추가되었는지)
print("Python 경로 목록:")
print("\n".join(sys.path))

Python 경로 목록:
/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master
/aiffel/storage/package
/opt/conda/lib/python39.zip
/opt/conda/lib/python3.9
/opt/conda/lib/python3.9/lib-dynload

/opt/conda/lib/python3.9/site-packages
/opt/conda/lib/python3.9/site-packages/IPython/extensions
/aiffel/.ipython
/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/models
/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/utils


In [3]:
# 기본 라이브러리
import os
import time
import shutil

# PyTorch 관련 라이브러리
import torch
import torch.utils.data as data
import torch.backends.cudnn as cudnn

# 이미지 변환 관련 라이브러리
from torchvision import transforms

# 데이터 로더 및 모델, 유틸리티 함수 불러오기
from data_loader import get_segmentation_dataset  # 데이터셋 불러오는 함수
from models.fast_scnn import get_fast_scnn  # Fast-SCNN 모델 불러오기
from utils.loss import MixSoftmaxCrossEntropyLoss, MixSoftmaxCrossEntropyOHEMLoss  # 손실 함수
from utils.lr_scheduler import LRScheduler  # 학습률 스케줄러
from utils.metric import SegmentationMetric  # 평가 지표

In [4]:
# 모델 및 학습 관련 하이퍼파라미터 설정
args = {
    "model": "fast_scnn",  # 사용할 모델 이름
    "dataset": "citys",  # 학습할 데이터셋 (예: Cityscapes)
    "base_size": 1024,  # 입력 이미지 기본 크기
    "crop_size": 768,  # 학습 시 사용할 크롭 크기
    "train_split": "train",  # 학습 데이터셋의 분할 방식

    # 학습 하이퍼파라미터
    "aux": False,  # 보조 손실 사용 여부
    "aux_weight": 0.4,  # 보조 손실 가중치
    "epochs": 10,  # 학습 에폭 수
    "start_epoch": 0,  # 학습 시작 에폭
    "batch_size": 2,  # 배치 크기
    "lr": 1e-2,  # 학습률
    "momentum": 0.9,  # 모멘텀
    "weight_decay": 1e-4,  # 가중치 감쇠 (L2 정규화)

    # 체크포인트 저장 위치
    "resume": True,  # 기존 모델 체크포인트 (사용하지 않음)
    "save_folder": "./weights",  # 모델 가중치 저장 경로

    # 평가 및 검증 설정
    "eval": True,  # 평가 모드 여부
    "no_val": False,  # 검증 생략 여부
}

# GPU 사용 여부 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True  # GPU 연산 최적화
args["device"] = device  # 학습에 사용할 디바이스 저장

In [7]:
train_img_path = "/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/datasets/citys/leftImg8bit/train"

# train 폴더 존재 여부 확인
print("📂 'train' 폴더 존재 여부:", os.path.exists(train_img_path))

# train 폴더 내부 파일 확인
if os.path.exists(train_img_path):
    print("📂 'train' 폴더 내부 파일:", os.listdir(train_img_path))

📂 'train' 폴더 존재 여부: True
📂 'train' 폴더 내부 파일: ['zurich', 'monchengladbach', 'strasbourg', 'weimar', 'erfurt', 'stuttgart', 'tubingen', 'ulm', 'cologne', 'krefeld', 'hamburg', 'aachen', 'jena', 'bremen', 'bochum', 'dusseldorf', 'darmstadt', 'hanover']


In [8]:
dataset_root = "/aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/datasets"

# 🔍 `gtFine` 폴더가 어디 있는지 확인
for root, dirs, files in os.walk(dataset_root):
    if "gtFine" in dirs:
        print(f"📂 Found gtFine folder at: {os.path.join(root, 'gtFine')}")

📂 Found gtFine folder at: /aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/datasets/citys/gtFine


In [9]:
# 이미지 전처리 변환 설정
input_transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 Tensor로 변환
    transforms.Normalize([.485, .456, .406], [.229, .224, .225]),  # 이미지 정규화
])

# 데이터셋 로드 (학습 및 검증)
data_kwargs = {"transform": input_transform, "base_size": args["base_size"], "crop_size": args["crop_size"]}
train_dataset = get_segmentation_dataset(args["dataset"], split=args["train_split"], mode="train", **data_kwargs)
val_dataset = get_segmentation_dataset(args["dataset"], split="val", mode="val", **data_kwargs)

# DataLoader 생성 (데이터 배치 단위로 로딩)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=args["batch_size"], shuffle=True, drop_last=True)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

# Fast-SCNN 모델 생성
model = get_fast_scnn(dataset=args["dataset"], aux=args["aux"])

# GPU 병렬 처리 지원 (멀티 GPU 사용 가능)
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=[0, 1, 2])

# 모델을 GPU 또는 CPU로 이동
model.to(args["device"])

Found 2975 images in /aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/datasets/citys/leftImg8bit/train
Found 500 images in /aiffel/aiffel/aiffel project model training/Fast-SCNN-pytorch-master/datasets/citys/leftImg8bit/val


FastSCNN(
  (learning_to_downsample): LearningToDownsample(
    (conv): _ConvBNReLU(
      (conv): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (dsconv1): _DSConv(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (dsconv2): _DSConv(
      (conv): Sequential(
        (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
        (1): BatchNorm2d(48, eps=

In [10]:
print(model)  # 모델이 정상적으로 선언되었는지 확인

FastSCNN(
  (learning_to_downsample): LearningToDownsample(
    (conv): _ConvBNReLU(
      (conv): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (dsconv1): _DSConv(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (dsconv2): _DSConv(
      (conv): Sequential(
        (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
        (1): BatchNorm2d(48, eps=

In [11]:
# 손실 함수 설정 (OHEM Loss 사용)
criterion = MixSoftmaxCrossEntropyOHEMLoss(aux=args["aux"], aux_weight=args["aux_weight"], ignore_index=-1).to(args["device"])

# 옵티마이저 (SGD 사용)
optimizer = torch.optim.SGD(model.parameters(), lr=args["lr"], momentum=args["momentum"], weight_decay=args["weight_decay"])

# 학습률 스케줄러 설정 (poly decay 사용)
lr_scheduler = LRScheduler(mode="poly", base_lr=args["lr"], nepochs=args["epochs"],
                           iters_per_epoch=len(train_loader), power=0.9)

# 평가 지표 (mIoU 등)
metric = SegmentationMetric(train_dataset.num_class)

# 최고 성능 저장을 위한 변수
best_pred = 0.0

w/ class balance


In [12]:
def train():
    global best_pred  # 최고 성능 비교를 위한 전역 변수 사용
    cur_iters = 0  # 현재 반복 횟수
    start_time = time.time()  # 학습 시작 시간

    # 각 에폭마다 학습 수행
    for epoch in range(args["start_epoch"], args["epochs"]):
        model.train()  # 모델을 학습 모드로 설정

        for i, (images, targets) in enumerate(train_loader):
            cur_lr = lr_scheduler(cur_iters)  # 현재 학습률 설정
            for param_group in optimizer.param_groups:
                param_group["lr"] = cur_lr  # 옵티마이저에 학습률 적용

            # 데이터를 GPU/CPU로 이동
            images, targets = images.to(args["device"]), targets.to(args["device"])

            # 모델 예측 및 손실 계산
            outputs = model(images)
            loss = criterion(outputs, targets)

            # 역전파 및 최적화 수행
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            cur_iters += 1  # 반복 횟수 증가
            if cur_iters % 10 == 0:  # 10회마다 로그 출력
                print(f"Epoch [{epoch}/{args['epochs']}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # 검증 수행 및 체크포인트 저장
        if not args["no_val"]:
            validation(epoch)

In [13]:
def validation(epoch):
    global best_pred  # 최고 성능 비교를 위한 전역 변수 사용
    model.eval()  # 모델을 평가 모드로 변경
    metric.reset()  # 평가 지표 초기화

    for i, (image, target) in enumerate(val_loader):
        image = image.to(args["device"])  # 이미지를 GPU/CPU로 이동

        with torch.no_grad():  # 그래디언트 계산 비활성화
            outputs = model(image)

        pred = torch.argmax(outputs[0], 1).cpu().data.numpy()  # 예측 결과 가져오기
        metric.update(pred, target.numpy())  # 평가 지표 업데이트

    pixAcc, mIoU = metric.get()  # 픽셀 정확도 및 mIoU 계산
    print(f"Epoch {epoch}, Validation PixAcc: {pixAcc:.3f}, mIoU: {mIoU:.3f}")

    new_pred = (pixAcc + mIoU) / 2  # 성능 평가
    if new_pred > best_pred:
        best_pred = new_pred

In [14]:
train()  # 학습 시작

Epoch [0/10], Step [9/1487], Loss: 1.6984
Epoch [0/10], Step [19/1487], Loss: 2.1239
Epoch [0/10], Step [29/1487], Loss: 2.0295
Epoch [0/10], Step [39/1487], Loss: 2.1423
Epoch [0/10], Step [49/1487], Loss: 1.4570
Epoch [0/10], Step [59/1487], Loss: 1.7141
Epoch [0/10], Step [69/1487], Loss: 1.4954
Epoch [0/10], Step [79/1487], Loss: 1.5687
Epoch [0/10], Step [89/1487], Loss: 2.0716
Epoch [0/10], Step [99/1487], Loss: 1.7081
Epoch [0/10], Step [109/1487], Loss: 2.5222
Epoch [0/10], Step [119/1487], Loss: 1.4102
Epoch [0/10], Step [129/1487], Loss: 1.8364
Epoch [0/10], Step [139/1487], Loss: 1.6245
Epoch [0/10], Step [149/1487], Loss: 2.1214
Epoch [0/10], Step [159/1487], Loss: 1.2951
Epoch [0/10], Step [169/1487], Loss: 1.9539
Epoch [0/10], Step [179/1487], Loss: 1.6137
Epoch [0/10], Step [189/1487], Loss: 1.8407
Epoch [0/10], Step [199/1487], Loss: 2.1681
Epoch [0/10], Step [209/1487], Loss: 1.5661
Epoch [0/10], Step [219/1487], Loss: 1.3354
Epoch [0/10], Step [229/1487], Loss: 1.0910

Epoch [1/10], Step [372/1487], Loss: 1.8904
Epoch [1/10], Step [382/1487], Loss: 1.4418
Epoch [1/10], Step [392/1487], Loss: 1.1142
Epoch [1/10], Step [402/1487], Loss: 1.1185
Epoch [1/10], Step [412/1487], Loss: 1.4657
Epoch [1/10], Step [422/1487], Loss: 1.0007
Epoch [1/10], Step [432/1487], Loss: 1.7595
Epoch [1/10], Step [442/1487], Loss: 1.4036
Epoch [1/10], Step [452/1487], Loss: 1.0686
Epoch [1/10], Step [462/1487], Loss: 1.3525
Epoch [1/10], Step [472/1487], Loss: 1.3537
Epoch [1/10], Step [482/1487], Loss: 1.0825
Epoch [1/10], Step [492/1487], Loss: 1.4741
Epoch [1/10], Step [502/1487], Loss: 1.6350
Epoch [1/10], Step [512/1487], Loss: 1.7159
Epoch [1/10], Step [522/1487], Loss: 1.5469
Epoch [1/10], Step [532/1487], Loss: 1.0567
Epoch [1/10], Step [542/1487], Loss: 1.5321
Epoch [1/10], Step [552/1487], Loss: 1.1166
Epoch [1/10], Step [562/1487], Loss: 3.7772
Epoch [1/10], Step [572/1487], Loss: 1.4862
Epoch [1/10], Step [582/1487], Loss: 1.6080
Epoch [1/10], Step [592/1487], L

KeyboardInterrupt: 